<a href="https://colab.research.google.com/github/khushi911/fish-classification-efficientdetlite2/blob/main/efficientdetlite2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 65.4 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudio2 amd64 19.6.0-1build1 [65.4 kB]
Fetched 65.4 kB in 0s (909 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Tele

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
# Install Kaggle API
!pip install -q kaggle
!pip install -q kaggle-cli

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 62.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB 393.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.2/147.2 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for lxml
  error: subprocess-exited-with-error
  
  × Running setup.py install for lxml did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: 

In [ ]:
!kaggle datasets download -d khushisarangamath/fishes --unzip

100% 136M/136M [00:03<00:00, 43.3MB/s]
100% 136M/136M [00:03<00:00, 36.6MB/s]


In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"khushisarangamath","key":"86f9ae71c5d59afa977219a98ae3d24b"}'}

In [ ]:
# only for google colab
import os
os.environ['KAGGLE_USERNAME'] = "khushisarangamath" 
os.environ['KAGGLE_KEY'] = "86f9ae71c5d59afa977219a98ae3d24b"

In [ ]:
!mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                             title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
themrityunjaypathak/covid-cases-and-deaths-worldwide            Covid Cases and Deaths WorldWide                     8KB  2023-02-01 12:22:51           3255         85  1.0              
ahsan81/hotel-reservations-classification-dataset               Hotel Reservations Dataset                         480KB  2023-01-04 12:50:31          12286        389  1.0              
datascientistanna/customers-dataset                             Shop Customer Data                                  23KB  2023-02-07 18:42:21           2380         69  1.0              
themrityunjaypathak/most-subscribed-1000-youtube-channels       M

In [ ]:
spec = model_spec.get('efficientdet_lite2')

In [ ]:
pwd

'/content'

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc('./train', './train', ['blackmolly', 'whitemolly', 'chocolatemolly', 'platy','guppy'])

In [ ]:
validation_data = object_detector.DataLoader.from_pascal_voc('./test', './test', ['blackmolly', 'whitemolly', 'chocolatemolly', 'platy','guppy'])

In [ ]:
model = object_detector.create(train_data, model_spec=spec, epochs=80, batch_size=8, train_whole_model=True, validation_data=validation_data)

Epoch 1/80
66/66 [==============================] - 115s 888ms/step - det_loss: 1.5883 - cls_loss: 1.0438 - box_loss: 0.0109 - reg_l2_loss: 0.0764 - loss: 1.6648 - learning_rate: 0.0090 - gradient_norm: 2.0043 - val_det_loss: 1.4719 - val_cls_loss: 1.0650 - val_box_loss: 0.0081 - val_reg_l2_loss: 0.0765 - val_loss: 1.5484
Epoch 2/80
66/66 [==============================] - 56s 846ms/step - det_loss: 0.9596 - cls_loss: 0.6105 - box_loss: 0.0070 - reg_l2_loss: 0.0765 - loss: 1.0361 - learning_rate: 0.0100 - gradient_norm: 3.3873 - val_det_loss: 0.7830 - val_cls_loss: 0.5299 - val_box_loss: 0.0051 - val_reg_l2_loss: 0.0766 - val_loss: 0.8596
Epoch 3/80
66/66 [==============================] - 57s 873ms/step - det_loss: 0.6929 - cls_loss: 0.4201 - box_loss: 0.0055 - reg_l2_loss: 0.0766 - loss: 0.7696 - learning_rate: 0.0100 - gradient_norm: 3.3654 - val_det_loss: 0.6675 - val_cls_loss: 0.3762 - val_box_loss: 0.0058 - val_reg_l2_loss: 0.0767 - val_loss: 0.7442
Epoch 4/80
66/66 [============

In [ ]:
model.evaluate(validation_data)

1/1 [==============================] - 20s 20s/step



{'AP': 0.6389539,
 'AP50': 0.9840471,
 'AP75': 0.79620254,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.6393246,
 'ARmax1': 0.62674665,
 'ARmax10': 0.72154397,
 'ARmax100': 0.7270995,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.7270995,
 'AP_/blackmolly': 0.59398836,
 'AP_/whitemolly': 0.635989,
 'AP_/chocolatemolly': 0.67831874,
 'AP_/platy': 0.7025844,
 'AP_/guppy': 0.5838893}

In [ ]:
model.export(export_dir='.',export_format=[ExportFormat.LABEL,ExportFormat.TFLITE])

In [ ]:
model.evaluate_tflite('model.tflite', validation_data)

60/60 [==============================] - 571s 10s/step



{'AP': 0.62202305,
 'AP50': 0.98624414,
 'AP75': 0.7527022,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.6221233,
 'ARmax1': 0.62293184,
 'ARmax10': 0.6847837,
 'ARmax100': 0.6862652,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.6862652,
 'AP_/blackmolly': 0.5772281,
 'AP_/whitemolly': 0.5915532,
 'AP_/chocolatemolly': 0.67600876,
 'AP_/platy': 0.6830249,
 'AP_/guppy': 0.5823003}